In [1]:
import torch
import os
import json
import tqdm

In [ ]:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de',tokenizer='moses', bpe='fastbpe',checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt')
en2de.eval()

en2de.cuda()

Using cache found in /home/zmykevin/.cache/torch/hub/pytorch_fairseq_master


In [ ]:
data_path = "/home/zmykevin/multilingual_VL/data/CC/annotations"
de_cc_path = '/'.join([data_path, "dataset_cc_de.json"])

with open(de_cc_path, "r") as f:
    cc_data_de = json.load(f)
print(len(cc_data_de["images"]))

In [14]:
data_path = "/home/zmykevin/multilingual_VL/data/CC/annotations"
valid_cc_annotation = '/'.join([data_path, "dataset_cc.json"])

with open(valid_cc_annotation, "r") as f:
    cc_data = json.load(f)

In [15]:
cc_data_origin = cc_data['images'].copy()
cc_imgs = cc_data['images']
print(cc_imgs[0])
# for img in cc_imgs:
#     caption = img[]
#     break
#cc_data_de = cc_data.copy()

{'imgid': 0, 'sentences': [{'tokens': ['a', 'very', 'typical', 'bus', 'station'], 'raw': 'a very typical bus station', 'deptree': [[4, 0], [2, 1], [4, 2], [4, 3], [-1, 4]]}], 'filename': '00000000.jpg', 'split': 'train'}


In [16]:
from progressbar import ProgressBar
import time

pbar = ProgressBar()

i = 0
total_elements = len(cc_imgs)


# for img in pbar(cc_imgs):
#     img_de = img.copy()
#     #translate the caption
#     caption_en = img_de['sentences'][0]['raw']
#     caption_de = en2de.translate(caption_en) 
#     tokens_de = caption_de.split()
#     #update img_de
#     img_de['sentences'][0]['tokens'] = tokens_de
#     img_de['sentences'][0]['raw'] = caption_de
#     cc_data_de['images'].append(img_de)
#     i += 1
#     print(i)
#     if i== 100:
#         break
batch_size = 32
cc_data_de = {"images":[], "dataset": cc_data['dataset']}
output_cc_data_path = '/'.join([data_path, "dataset_cc_de.json"])

start_time = time.perf_counter()
for i in range(0,total_elements,batch_size):
    #form the batch of sentences
    captions_en_batch = [x['sentences'][0]['raw'] for x in cc_imgs[i:i+batch_size]]
    #print(captions_en_batch)
    captions_de_batch = en2de.translate(captions_en_batch)
    #print(captions_de_batch)
    for j in range(i,i+batch_size):
        cc_data['images'][j]['sentences'][0]['raw'] = captions_de_batch[j-i]
        cc_data['images'][j]['sentences'][0]['tokens'] = captions_de_batch[j-i].split()
#         print(captions_de_batch[j-i])
#         print(cc_data['images'][j]['sentences'])
#         print(j-i)
        cc_data_de['images'].append(cc_data['images'][j])
    
    #save cc_data_de
    if i == 0:
        with open(output_cc_data_path, "w") as f_out:
            json.dump(cc_data_de, f_out, indent=4, sort_keys=True)
        break
#print(cc_data_de["images"][0])
print("Finish translation of 300 data in {} seconds".format(time.perf_counter()-start_time))

#save the new data

# with open(output_cc_data_path, "w") as f_out:
#     json.dump(cc_data_de, f_out, indent=4, sort_keys=True)

['eine sehr typische Bushaltestelle', 'Sierra sah in diesem Top und diesem Rock umwerfend aus, während sie mit einer Person an ihrer ehemaligen Universität auftrat', 'junges verwirrtes Mädchen steht vor einem Kleiderschrank', 'Innenarchitektur des modernen Wohnzimmers mit Kamin in einem neuen Haus', 'kybernetische Szene isoliert auf weißem Hintergrund.', 'Gangsta-Rap-Künstler besucht Sportteam gegen Playoff-Spiel im Bezirk.', 'der Steg: verschiedene Arten von Pflanzen, um eine Vielzahl von Ökosystemen zu etablieren.', 'traditionelle dekorative Blumen Paisley Bandanna.', '# der Sportmannschaft läuft während des Spiels gegen die Sportmannschaft.', 'nach geographischen Merkmalen oder in der Stadt - eine Kuppel für jede Umgebung', 'Ein Flug war unterwegs, als das Tier in der Nacht zum Dienstag frei kam', 'Obwohl die landwirtschaftlichen Bedingungen für den Tabakanbau nicht ideal sind, gibt es eine einheimische Produktion.', 'us state spricht während einer Demonstration am Donnerstag.', 'Sc